### --- Day 20: Pulse Propagation ---

With your help, the Elves manage to find the right parts and fix all of the machines. Now, they just need to send the command to boot up the machines and get the sand flowing again.

The machines are far apart and wired together with long cables. The cables don't connect to the machines directly, but rather to communication modules attached to the machines that perform various initialization tasks and also act as communication relays.

Modules communicate using pulses. Each pulse is either a high pulse or a low pulse. When a module sends a pulse, it sends that type of pulse to each module in its list of destination modules.

There are several different types of modules:

Flip-flop modules (prefix %) are either on or off; they are initially off. If a flip-flop module receives a high pulse, it is ignored and nothing happens. However, if a flip-flop module receives a low pulse, it flips between on and off. If it was off, it turns on and sends a high pulse. If it was on, it turns off and sends a low pulse.

Conjunction modules (prefix &) remember the type of the most recent pulse received from each of their connected input modules; they initially default to remembering a low pulse for each input. When a pulse is received, the conjunction module first updates its memory for that input. Then, if it remembers high pulses for all inputs, it sends a low pulse; otherwise, it sends a high pulse.

There is a single broadcast module (named broadcaster). When it receives a pulse, it sends the same pulse to all of its destination modules.

Here at Desert Machine Headquarters, there is a module with a single button on it called, aptly, the button module. When you push the button, a single low pulse is sent directly to the broadcaster module.

After pushing the button, you must wait until all pulses have been delivered and fully handled before pushing it again. Never push the button if modules are still processing pulses.

Pulses are always processed in the order they are sent. So, if a pulse is sent to modules a, b, and c, and then module a processes its pulse and sends more pulses, the pulses sent to modules b and c would have to be handled first.

The module configuration (your puzzle input) lists each module. The name of the module is preceded by a symbol identifying its type, if any. The name is then followed by an arrow and a list of its destination modules. For example:
```
broadcaster -> a, b, c
%a -> b
%b -> c
%c -> inv
&inv -> a
```
In this module configuration, the broadcaster has three destination modules named a, b, and c. Each of these modules is a flip-flop module (as indicated by the % prefix). a outputs to b which outputs to c which outputs to another module named inv. inv is a conjunction module (as indicated by the & prefix) which, because it has only one input, acts like an inverter (it sends the opposite of the pulse type it receives); it outputs to a.

By pushing the button once, the following pulses are sent:
```
button -low-> broadcaster
broadcaster -low-> a
broadcaster -low-> b
broadcaster -low-> c
a -high-> b
b -high-> c
c -high-> inv
inv -low-> a
a -low-> b
b -low-> c
c -low-> inv
inv -high-> a
```
After this sequence, the flip-flop modules all end up off, so pushing the button again repeats the same sequence.

Here's a more interesting example:
```
broadcaster -> a
%a -> inv, con
&inv -> b
%b -> con
&con -> output
```
This module configuration includes the broadcaster, two flip-flops (named a and b), a single-input conjunction module (inv), a multi-input conjunction module (con), and an untyped module named output (for testing purposes). The multi-input conjunction module con watches the two flip-flop modules and, if they're both on, sends a low pulse to the output module.

Here's what happens if you push the button once:
```
button -low-> broadcaster
broadcaster -low-> a
a -high-> inv
a -high-> con
inv -low-> b
con -high-> output
b -high-> con
con -low-> output
```
Both flip-flops turn on and a low pulse is sent to output! However, now that both flip-flops are on and con remembers a high pulse from each of its two inputs, pushing the button a second time does something different:
```
button -low-> broadcaster
broadcaster -low-> a
a -low-> inv
a -low-> con
inv -high-> b
con -high-> output
```
Flip-flop a turns off! Now, con remembers a low pulse from module a, and so it sends only a high pulse to output.

Push the button a third time:
```
button -low-> broadcaster
broadcaster -low-> a
a -high-> inv
a -high-> con
inv -low-> b
con -low-> output
b -low-> con
con -high-> output
```
This time, flip-flop a turns on, then flip-flop b turns off. However, before b can turn off, the pulse sent to con is handled first, so it briefly remembers all high pulses for its inputs and sends a low pulse to output. After that, flip-flop b turns off, which causes con to update its state and send a high pulse to output.

Finally, with a on and b off, push the button a fourth time:
```
button -low-> broadcaster
broadcaster -low-> a
a -low-> inv
a -low-> con
inv -high-> b
con -high-> output
```
This completes the cycle: a turns off, causing con to remember only low pulses and restoring all modules to their original states.

To get the cables warmed up, the Elves have pushed the button 1000 times. How many pulses got sent as a result (including the pulses sent by the button itself)?

In the first example, the same thing happens every time the button is pushed: 8 low pulses and 4 high pulses are sent. So, after pushing the button 1000 times, 8000 low pulses and 4000 high pulses are sent. Multiplying these together gives 32000000.

In the second example, after pushing the button 1000 times, 4250 low pulses and 2750 high pulses are sent. Multiplying these together gives 11687500.

Consult your module configuration; determine the number of low pulses and high pulses that would be sent after pushing the button 1000 times, waiting for all pulses to be fully handled after each push of the button. What do you get if you multiply the total number of low pulses sent by the total number of high pulses sent?


In [77]:
f = open("input.txt", "r")

circuit = {}
for line in f:
    line = line.strip().split(' -> ')
    if line[0][0] == '%' or line[0][0] == '&':
        circuit[line[0][1:]] = {'type': line[0][0], 'out': list(line[1].strip().split(', ')), 'in':[], 'state':0 }
    else:
        circuit[line[0]] = {'type':'b', 'out':list(line[1].strip().split(', ')), 'in':[]}
        
for name in circuit:
    spec = circuit[name]
    for output in spec['out']:
        if output in circuit:
            circuit[output]['in'].append(name)
        
for name in circuit:
    if circuit[name]['type'] == '&':
        circuit[name]['memory'] = {}
        
        for inp in circuit[name]['in']:
            circuit[name]['memory'][inp] = 0 
            
def send_pulse_button(times = 1):
    total_low = 0
    total_high = 0
    
    for i in range(times):
        queue = [(0, 'button', 'broadcaster')]
        
        next_queue = []
        while len(queue) > 0: 
            signal, source, name = queue.pop(0)

            if name == 'rx' and signal == 0:
                print(i)
            
            # Count here
            if signal == 1:
                total_high += 1
            else:
                total_low += 1

            if name not in circuit:
                continue
            spec = circuit[name]

            if spec['type'] == '%':
                if signal == 0:
                    spec['state'] = 0 if spec['state'] == 1 else 1

                    for n in spec['out']:
                        queue.append((spec['state'], name, n))

            elif spec['type'] == '&':
                spec['memory'][source] = signal
                signal_to_send = 0
                for inp in spec['in']:
                    if spec['memory'][inp] == 0:
                        signal_to_send = 1
                        break

                for n in spec['out']:
                    queue.append((signal_to_send, name, n))
            else:
                for n in spec['out']:
                    queue.append((signal, name, n))

    print(total_low, total_high, total_high*total_low)

send_pulse_button(100000000)

KeyboardInterrupt: 

### --- Part Two ---

The final machine responsible for moving the sand down to Island Island has a module attached named rx. The machine turns on when a single low pulse is sent to rx.

Reset all modules to their default states. Waiting for all pulses to be fully handled after each button press, what is the fewest number of button presses required to deliver a single low pulse to the module named rx?


In [75]:
circuit

{'nr': {'type': '%', 'out': ['hq'], 'in': ['sz', 'ft'], 'state': 1},
 'xk': {'type': '%', 'out': ['sn'], 'in': ['cl', 'lk'], 'state': 1},
 'cl': {'type': '%', 'out': ['xk'], 'in': ['gm', 'lk'], 'state': 1},
 'mj': {'type': '%', 'out': ['dq', 'qr'], 'in': ['jj'], 'state': 1},
 'gm': {'type': '%',
  'out': ['lk', 'cl'],
  'in': ['broadcaster', 'lk'],
  'state': 0},
 'mv': {'type': '%', 'out': ['lz', 'rr'], 'in': ['nk'], 'state': 1},
 'qr': {'type': '&',
  'out': ['cz', 'sp', 'lb', 'xt', 'fx'],
  'in': ['mj', 'rv', 'vl', 'jj', 'cz', 'dq', 'bz', 'mb', 'zj'],
  'state': 0,
  'memory': {'mj': 1,
   'rv': 0,
   'vl': 1,
   'jj': 0,
   'cz': 0,
   'dq': 0,
   'bz': 1,
   'mb': 0,
   'zj': 0}},
 'xt': {'type': '%', 'out': ['vl'], 'in': ['qr', 'cz'], 'state': 1},
 'dd': {'type': '&',
  'out': ['rx'],
  'in': ['nx', 'sp', 'cc', 'jq'],
  'state': 0,
  'memory': {'nx': 0, 'sp': 0, 'cc': 0, 'jq': 0}},
 'rv': {'type': '%', 'out': ['qr'], 'in': ['dq'], 'state': 0},
 'ts': {'type': '%',
  'out': ['lz',

In [107]:
f = open("input.txt", "r")

circuit = {}
for line in f:
    line = line.strip().split(' -> ')
    if line[0][0] == '%' or line[0][0] == '&':
        circuit[line[0][1:]] = {'type': line[0][0], 'out': list(line[1].strip().split(', ')), 'in':[], 'state':0 }
    else:
        circuit[line[0]] = {'type':'b', 'out':list(line[1].strip().split(', ')), 'in':[]}
        
for name in circuit:
    spec = circuit[name]
    for output in spec['out']:
        if output in circuit:
            circuit[output]['in'].append(name)
        
for name in circuit:
    if circuit[name]['type'] == '&':
        circuit[name]['memory'] = {}
        
        for inp in circuit[name]['in']:
            circuit[name]['memory'][inp] = 0 
            
def send_pulse_button(times = 1):
    total_low = 0
    total_high = 0
    
    updates = []
    
    for i in range(times):
        updates.append([])
        queue = [(0, 'button', 'broadcaster')]
        
        next_queue = []
        while len(queue) > 0: 
            signal, source, name = queue.pop(0)

            if name == 'dd':
                updates[i].append((signal, source))
            
            # Count here
            if signal == 1:
                total_high += 1
            else:
                total_low += 1

            if name not in circuit:
                continue
            spec = circuit[name]

            if spec['type'] == '%':
                if signal == 0:
                    spec['state'] = 0 if spec['state'] == 1 else 1

                    for n in spec['out']:
                        queue.append((spec['state'], name, n))

            elif spec['type'] == '&':
                spec['memory'][source] = signal
                signal_to_send = 0
                for inp in spec['in']:
                    if spec['memory'][inp] == 0:
                        signal_to_send = 1
                        break

                for n in spec['out']:
                    queue.append((signal_to_send, name, n))
            else:
                for n in spec['out']:
                    queue.append((signal, name, n))
                    
        
    print(total_low, total_high, total_high*total_low)
    
    register = []
    for update in updates:
        register.append({'sp': [0,0], 'cc': [0,0], 'nx': [0,0], 'jq': [0,0]})
        for (signal, source) in update:
            register[-1][source][signal] += 1
    return register

register = send_pulse_button(10000)

178155 493171 87860879505


In [109]:
# Cycle Finding for sp
for name in ['sp', 'cc', 'nx', 'jq']:
    for i in range(1, len(register)//2 + 1):
        cycle = []

        for j in range(i):
            cycle.append(register[j][name])

        cycle_present = True
        for k in range(len(register)):
            if register[k][name] != cycle[k % i]:
                cycle_present = False
                break

        if cycle_present:
            print("For ", name, " found cycle of length ", i)

For  sp  found cycle of length  4013
For  cc  found cycle of length  4001
For  nx  found cycle of length  3851
For  jq  found cycle of length  3911


In [111]:
# It's not guaranteed that this is the correct answer as there may be multiple places where the modules turn on
# However, if they only turn on once this is the only answer
4013*4001*3851*3911

241823802412393